In [1]:
# Import libraries and packages
import pandas as pd
import numpy as np
import warnings
import os
warnings.filterwarnings("ignore")

In [2]:
# Load filtered meds
meds = pd.read_csv('/opt/moredata/dchanci/pediatric_sepsis/data_screening/filtered_meds.csv')
meds[['dob', 'mar_time']] = meds[['dob', 'mar_time']].apply(pd.to_datetime)

# Load antiinfective meds
antiinf = pd.read_csv('/opt/moredata/dchanci/pediatric_sepsis/data_screening/antiinf_meds.csv')
antiinf[['dob', 'mar_time']] = antiinf[['dob', 'mar_time']].apply(pd.to_datetime)

# Concat
meds = pd.concat([meds, antiinf])
meds = meds[~(meds['mar_action'].str.contains('downtime', case=False))]

In [3]:
# Load antiinfective meds
antiinf_list = pd.read_csv('../files/Emory_antiinfective_meds.csv')
antiinf_list = antiinf_list['med_name'].unique().tolist()
antiinf_list = [x.split(' ')[0] for x in antiinf_list]
antiinf_list = [x.split('-')[0] for x in antiinf_list]

In [4]:
# Create flags
asthma_list = ['albuterol', 'dexamethasone', 'epinephrine', 'methylprednisolone', 'magnesium sulfate', 'terbutaline', 'levalbuterol', 'xopenex']
meds['asthma_meds'] = 0
meds.loc[(meds['med'].str.contains('|'.join(asthma_list), case=False)) &
        ~(meds['med'].str.contains('norepinephrine', case=False)), 'asthma_meds'] = 1

seizures_list = ['lorazepam', 'levetiracetam', 'fosphenytoin', 'phenobarbital']
meds['seizure_meds'] = 0
meds.loc[meds['med'].str.contains('|'.join(seizures_list), case=False), 'seizure_meds'] = 1

vasopressors_list = ['epinephrine', 'phenylephrine', 'dopamine', 'norepinephrine', 'vasopressin']
meds['vasopressors'] = 0
meds.loc[meds['med'].str.contains('|'.join(vasopressors_list), case=False), 'vasopressors'] = 1

meds['antiinfective_meds'] = 0
meds.loc[meds['med'].str.contains('|'.join(antiinf_list), case=False), 'antiinfective_meds'] = 1

meds['insulin'] = 0
meds.loc[meds['med'].str.contains('insulin', case=False), 'insulin'] = 1

# Save data
meds.to_csv('/opt/moredata/dchanci/pediatric_sepsis/data_models/filtered_meds.csv', index=False)